 souboru s daty na Slacku máš data o hodnotách finančních indikátorů 100 největších společností obchodovaných na americké burze. Naším cílem je zjistit, které indikátory nejvíce ovlivňují cenu, a vytvořit model, který odhadne cenu akcie na základě hodnot finančních indikátorů.


In [4]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
data = pd.read_csv("df.csv")
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical


Jeden z indikátorů je označený jako Y a je poměrem ceny a účetní hodnoty akcie. Hodnota indikátoru je odrazem ceny akcie. Ostatní indikátory jso následující:

běžná likvidita (Current Ratio, CR),
zadluženost (Debt to Assets, DA),
finanční páka (Financial Leverage, FL)
provozní zisková marže (Operating Profit Margin, OPM),
obrat pohledávek (Receivables Turnover, RT),
obrat celkových aktiv (Total Assets Turnover, TAT).
Úkol můžeš odevzdat jako Jupyter notebook.



Část 1
Vytvoř korelační matici a podívej se, který ukazatel má největší vliv na indikátor Y akcie.


In [3]:
data_corr = data.drop(columns=["Company", "Sector"])
data_corr.corr()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


Část 2
Vytvoř regresní model, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu vlož hodnoty ostatních indikátorů jako vysvětlující proměnné.

In [11]:
from statsmodels.stats.outliers_influence import OLSInfluence

formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data)
results = mod.fit()
influence = OLSInfluence(results)
data['Cook Distance'] = influence.cooks_distance[0]
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Cook Distance
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.000031
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,0.035577
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,0.000286
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,0.025369
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,0.000151


In [12]:
data["Sector"].unique()


array(['Industrials', 'Communication Services', 'Consumer Defensive',
       'Consumer Cyclical', 'Healthcare', 'Basic Materials', 'Technology',
       'Energy', 'Conglomerates', 'Financial Services'], dtype=object)

U žádných hodnot nemám cookovu vzdálenost větší než 1, takže ponechávám

S využitím modulu statsmodels vytvoř regresní model a zobraz si tabulku se souhrnem významů. Podívej se na hodnoty koeficientů a na výsledky testu statistické významnosti koeficientů. Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestav nový model bez tohoto koeficientů (případně beze všech nevýznamných koeficientů).

In [13]:
data = data[data['Cook Distance'] < 1]
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     24.47
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           1.88e-19
Time:                        22:42:25   Log-Likelihood:                -364.33
No. Observations:                  98   AIC:                             746.7
Df Residuals:                      89   BIC:                             769.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.7147      5.415     -1.055      0.294     -16.473       5.044
CR             1.3997      1.198      1.169      0.246      -0.980       3.780
DA           -16.8499      9.739     -1.730      0.087     -36.202       2.502
FL             0.8305      1.236      0.672      0.503      -1.625       3.286
OPM           22.1704      7.858      2.821      0.006       6.557      37.784
RT            -0.0377      0.067     -0.561      0.576      -0.171       0.096
TAT            8.8368      2.713      3.257      0.002       3.445      14.228
DE             4.3384      2.572      1.687      0.095      -0.772       9.449
PT            -0.0126      0.094     -0.134      0.894      -0.200       0.175
==============================================================================
Omnibus:                       20.515   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.817
Skew:                           0.689   Prob(JB):                     5.60e-12
Kurtosis:                       6.285   Cond. No.                         280.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pro statistickou významnost se dívám na p hodnotu. Pokud je p-hodnota nižší než 0,05, obvykle se považuje za statisticky významné.

Jako statisticky významné hodnotíme ukazatele:
OPM a TAT
provozní zisková marže (Operating Profit Margin, OPM),
obrat celkových aktiv (Total Assets Turnover, TAT).

Nyní tedy vytvořím model, kde budu uvažovat pouze tyto statisticky významné ukazatele.

In [57]:
data = pd.read_csv("df.csv")
data = data[["Y", "OPM", "TAT", "DE", "DA", "CR", "Company"]]
data.head()

,Y,OPM,TAT,DE,DA,CR,Company
0,1.95,0.0770,0.40,0.43,0.20,1.19,2Q3DU
1,7.26,0.2037,0.36,6.52,0.64,0.29,46JNq
2,7.78,0.3055,0.72,0.06,0.04,2.93,4ISOt
3,6.17,0.0708,1.80,0.88,0.25,1.03,5AWwu
4,0.00,0.1677,0.93,0.00,0.47,1.20,5hbZR


In [58]:
formula = "Y ~ OPM + TAT + DE + DA + CR"
mod = smf.ols(formula=formula, data=data)
results = mod.fit()

In [59]:



# Calculate Cook's Distance for the new model
influence = OLSInfluence(results)
data["Cook Distance"] = influence.cooks_distance[0]

# Display the updated DataFrame with Cook's Distance
data.head()

,Y,OPM,TAT,DE,DA,CR,Company,Cook Distance
0,1.95,0.0770,0.40,0.43,0.20,1.19,2Q3DU,0.000001
1,7.26,0.2037,0.36,6.52,0.64,0.29,46JNq,0.038343
2,7.78,0.3055,0.72,0.06,0.04,2.93,4ISOt,0.001229
3,6.17,0.0708,1.80,0.88,0.25,1.03,5AWwu,0.008060
4,0.00,0.1677,0.93,0.00,0.47,1.20,5hbZR,0.000108


In [60]:
data = data[data["Cook Distance"] < 1]

In [61]:

mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.686
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     40.60
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           5.96e-22
Time:                        23:19:55   Log-Likelihood:                -368.02
No. Observations:                  99   AIC:                             748.0
Df Residuals:                      93   BIC:                             763.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.7095      3.908     -0.949      0.345     -11.469       4.050
OPM           22.0798      7.445      2.966      0.004       7.295      36.865
TAT            8.0989      2.121      3.818      0.000       3.886      12.311
DE             6.0659      0.480     12.632      0.000       5.112       7.019
DA           -20.4576      7.236     -2.827      0.006     -34.827      -6.088
CR             1.3108      1.135      1.155      0.251      -0.944       3.565
==============================================================================
Omnibus:                       19.694   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.939
Skew:                           0.676   Prob(JB):                     6.42e-11
Kurtosis:                       6.091   Cond. No.                         23.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Podle p-value jsem měla mít v modelu jen TAT a OPM. Ale koeficient determinace se mi pak snížil oproti původnímu modelu. Nakonec jsem vybrala hodnoty "Y ~ OPM + TAT + DE + DA + CR"



Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. 

In [62]:
predicted_values = results.predict(data)
data["Predicted_Y"] = predicted_values
data

C:\Users\Admin\AppData\Local\Temp\ipykernel_1872\2006058518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Predicted_Y'] = predicted_values


,Y,OPM,TAT,DE,DA,CR,Company,Cook Distance,Predicted_Y
0,1.95,0.0770,0.40,0.43,0.20,1.19,2Q3DU,0.000001,2.121363
1,7.26,0.2037,0.36,6.52,0.64,0.29,46JNq,0.038343,25.734428
2,7.78,0.3055,0.72,0.06,0.04,2.93,4ISOt,0.001229,11.898194
3,6.17,0.0708,1.80,0.88,0.25,1.03,5AWwu,0.008060,15.977513
4,0.00,0.1677,0.93,0.00,0.47,1.20,5hbZR,0.000108,1.409409
...,...,...,...,...,...,...,...,...,...
94,5.05,0.2574,0.46,0.87,0.31,1.27,y6kqD,0.000035,6.302694
95,5.18,0.2557,0.41,0.85,0.37,1.50,yhCH3,0.000005,4.741885
96,7.40,0.2363,0.64,0.69,0.27,0.70,YqdX2,0.000006,6.976543
97,5.99,0.2576,0.51,0.28,0.12,1.49,yUjyz,0.000103,7.577957


Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená).

In [65]:
data["difference_Y"] = data["Predicted_Y"] - data["Y"]
data

C:\Users\Admin\AppData\Local\Temp\ipykernel_1872\1063899112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["difference_Y"] = data["Predicted_Y"] - data["Y"]


,Y,OPM,TAT,DE,DA,CR,Company,Cook Distance,Predicted_Y,difference_Y
0,1.95,0.0770,0.40,0.43,0.20,1.19,2Q3DU,0.000001,2.121363,0.171363
1,7.26,0.2037,0.36,6.52,0.64,0.29,46JNq,0.038343,25.734428,18.474428
2,7.78,0.3055,0.72,0.06,0.04,2.93,4ISOt,0.001229,11.898194,4.118194
3,6.17,0.0708,1.80,0.88,0.25,1.03,5AWwu,0.008060,15.977513,9.807513
4,0.00,0.1677,0.93,0.00,0.47,1.20,5hbZR,0.000108,1.409409,1.409409
...,...,...,...,...,...,...,...,...,...,...
94,5.05,0.2574,0.46,0.87,0.31,1.27,y6kqD,0.000035,6.302694,1.252694
95,5.18,0.2557,0.41,0.85,0.37,1.50,yhCH3,0.000005,4.741885,-0.438115
96,7.40,0.2363,0.64,0.69,0.27,0.70,YqdX2,0.000006,6.976543,-0.423457
97,5.99,0.2576,0.51,0.28,0.12,1.49,yUjyz,0.000103,7.577957,1.587957


In [67]:
data_sorted = data.sort_values(by="difference_Y", ascending=False)
data_sorted

,Y,OPM,TAT,DE,DA,CR,Company,Cook Distance,Predicted_Y,difference_Y
79,6.97,0.0631,0.41,6.93,0.37,0.76,SYXLy,0.064297,31.041969,24.071969
6,3.06,-0.0415,0.36,6.69,0.49,1.19,6qLRA,0.069149,25.410776,22.350776
1,7.26,0.2037,0.36,6.52,0.64,0.29,46JNq,0.038343,25.734428,18.474428
56,11.30,0.0349,3.24,0.49,0.14,1.00,n2wGj,0.179512,28.551333,17.251333
89,4.31,0.7522,0.72,0.05,0.03,1.76,Wjagj,0.098710,19.731266,15.421266
...,...,...,...,...,...,...,...,...,...,...
54,31.06,0.1212,0.87,0.23,0.11,1.38,mS0GZ,0.020872,8.109132,-22.950868
81,30.36,0.0436,0.55,0.43,0.15,1.05,tX5FA,0.040062,3.698701,-26.661299
75,95.58,0.1384,1.87,11.29,0.53,1.23,QiwB1,0.288482,65.021505,-30.558495
39,49.43,0.5339,0.50,1.90,0.37,1.29,hqTVU,0.116402,16.013450,-33.416550


Koupila bych si akcie firmy SYXLy , kdybych věděla název :-)
Ted už to jenom nahrát..